In [ ]:
''' This file will be removed before merging with master'''

In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import math
from pprint import pprint
import holidays
LATEST_DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'

def load_dataset(url):
    latest_df = pd.read_csv(url,
                            parse_dates=['Date'],
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str,
                                   "RegionCode": str},
                            error_bad_lines=False)
    latest_df["RegionName"] = latest_df["RegionName"].fillna("")
    return latest_df

In [ ]:
df = load_dataset(LATEST_DATA_URL)
df.sample(3)
df[df["RegionName"]== ""]

In [ ]:
pprint(df.columns)
all_country_codes = sorted(list(set( [(x,y) for x,y in  zip(df['CountryCode'].to_list(), df['CountryName'].to_list())])))  
##print(len(all_country_codes))
#pprint(all_country_codes) 
with open('country_codes.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in all_country_codes))

In [ ]:
region_info = set([(x,y,z,w) for x,y,z,w in zip(df['RegionCode'].to_list(), df['CountryCode'].to_list(), df['CountryName'].to_list(), df['RegionName'].to_list()) if isinstance(x, str)])

region_info = sorted(list(region_info))

with open('region_codes.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s' % x for x in region_info))

In [ ]:
country_code = "GBR"
df[(df["CountryCode"]==country_code) & (df["Jurisdiction"]== "NAT_TOTAL") & 
   (pd.Timestamp('2020-01-01')<=df["Date"]) &
   (df["Date"]<=pd.Timestamp('2020-03-01'))]   [["Date", "ConfirmedCases", "RegionCode"]]

In [47]:
true = []
for x,y in all_country_codes:
    if x in dir(holidays):
        if x == "MAR":
             x = "MOR"
        f = getattr(holidays, x) 
        true.append((x, y, " 12/25/2020" in f())) 

print(len(true))
pprint(true)


66
[('ABW', 'Aruba', True),
 ('AGO', 'Angola', True),
 ('ARE', 'United Arab Emirates', False),
 ('ARG', 'Argentina', True),
 ('AUS', 'Australia', True),
 ('AUT', 'Austria', True),
 ('BDI', 'Burundi', True),
 ('BEL', 'Belgium', True),
 ('BGD', 'Bangladesh', False),
 ('BLR', 'Belarus', True),
 ('BRA', 'Brazil', True),
 ('CAN', 'Canada', True),
 ('CHE', 'Switzerland', True),
 ('CHL', 'Chile', True),
 ('COL', 'Colombia', True),
 ('CZE', 'Czech Republic', True),
 ('DEU', 'Germany', True),
 ('DJI', 'Djibouti', False),
 ('DNK', 'Denmark', True),
 ('DOM', 'Dominican Republic', True),
 ('EGY', 'Egypt', False),
 ('ESP', 'Spain', True),
 ('EST', 'Estonia', True),
 ('FIN', 'Finland', True),
 ('FRA', 'France', True),
 ('GBR', 'United Kingdom', True),
 ('GRC', 'Greece', True),
 ('HKG', 'Hong Kong', True),
 ('HND', 'Honduras', True),
 ('HRV', 'Croatia', True),
 ('HUN', 'Hungary', True),
 ('IND', 'India', True),
 ('IRL', 'Ireland', True),
 ('ISL', 'Iceland', True),
 ('ISR', 'Israel', False),
 ('ITA', 

In [ ]:
country_code = "GBR"
df1 = df[(df["RegionCode"]=="BR_MA") ]   
df2 = df1[(pd.Timestamp('2020-01-01')<=df1["Date"])]   
df3 = df2[(df2["Date"]<=pd.Timestamp('2020-03-01'))] 
df3['Date'] = df3['Date'].apply(lambda x: x.to_pydatetime().strftime("%Y%m%d"))
df4 = df3[["Date", "ConfirmedCases"]] 


In [ ]:
df4

In [ ]:
import datetime
datetime.date.today().strftime("%Y-%m-%d")

In [ ]:
df4

In [64]:
import os
import pandas as pd
import argparse
import datetime
import holidays
from collections import defaultdict
from pprint import pprint
from dateutil.parser import parse

LATEST_DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'


def load_dataset(url):
    latest_df = pd.read_csv(url,
                            parse_dates=['Date'],
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str,
                                   "RegionCode": str},
                            error_bad_lines=False)
    latest_df["RegionName"] = latest_df["RegionName"].fillna("")
    return latest_df


# get country codes and save to file for reference
oxford_df = load_dataset(LATEST_DATA_URL)
country_codes = set([(x,y) for x,y in
                     zip(oxford_df['CountryCode'].to_list(),
                         oxford_df['CountryName'].to_list()
                        )])
country_codes = sorted(list(country_codes))
with open('country_codes.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in country_codes))


# get region codes and save to file for reference
region_codes = set([(x,y,z,w) for x,y,z,w in
                   zip(oxford_df['RegionCode'].to_list(),
                       oxford_df['CountryCode'].to_list(),
                       oxford_df['CountryName'].to_list(),
                       oxford_df['RegionName'].to_list()) if isinstance(x, str)])
region_codes = sorted(list(region_codes))

with open('region_codes.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s' % x for x in region_codes))


# Key for weekdays/weekend
#  Mon   Tue   Wed   Thu   Fri   Sat   Sun
#   0     1     2     3     4     5     6
WEEKENDS = defaultdict(lambda : (5,6))
WEEKENDS.update({
    "AFG": (3,4),   # Afghanistan
    "DZA": (4,5),   # Algeria
    "BHR": (4,5),   # Bahrain
    "BGD": (4,5),   # Bangladesh
    "BRN": (4,6),   # Brunei
    "DJI": (None,4),    # Djibouti
    "EGY": (4,5),   # Egypt
    "GIN": (None,6),    # Eq, Guinea
    "HKG": (None,6),    #Hong Kong
    "IRN": (None,4),    # Iran
    "IRQ": (4,5),   # Iraq
    "ISR": (4,5),   # Israel
    "JOR": (4,5),   # Jordan
    "KWT": (4,5),   # Kuwait
    "LBY": (4,5),   # Libya
    "NPL": (None,5),    # Nepal
    "OMN": (4,5),   # Oman
    "PSE": (None,4),    # Palestine
    "QAT": (4,5),   # Qatar
    "SAU": (4,5),   # Saudi Arabia
    "SOM": (None,4),    # Somalia
    "SDN": (4,5),   # Sudan
    "SYR": (4,5),   # Syria
    "ARE": (4,5),   # UAE
    "UGA": (None,6),    # Uganda
    "YEM": (4,5),   # Yemen
    })

def holiday_country(country, date): 

    # the two anomalies in code in holidays w.r.t oxford country codes
    # case for morocco and bulgaria
    if country == "MAR":
        country == "MOR"
    if country == "BGR":
        country == "BLG"

    if country in holidays.list_supported_countries(): 
        func = getattr(holidays, country) 
        if date in func():
            return 2  # return holiday (priority over weekend)
          
    return int( parse(date).weekday() in WEEKENDS[country])

def holiday_region(region, date):
    country, reg = region.split("_")

    if country == "BR":
        country == "BRA"
    elif country == "US":
        country == "USA"
    elif country == "UK":
        country == "GBR"
 
    func = getattr(holidays, country)
    if date in func(state=reg):  # holiday for the state
        return 2  # return holiday (priority over weekend)
          
    return int( parse(date).weekday() in WEEKENDS[country])  # check for weekend

country_code = "IND"
start_date = "2020-1-1"
end_date = "2020-4-1"
country_name = [y for x, y in country_codes if x == country_code][0]
region_name = "nan"

df_final = oxford_df[(oxford_df["CountryCode"]==country_code)]
# for three countries with sub-regions look for only the national sum
if country_code in ["USA", "GBR", "BRA"]:
    df_final = df_final[(df_final["Jurisdiction"]== "NAT_TOTAL")]

# filter between dates
df_final = df_final[(pd.Timestamp(start_date)<=df_final["Date"])]
df_final = df_final[(df_final["Date"]<=pd.Timestamp(end_date))]
 
df_final['Date']=df_final['Date'].astype(str)
df_final["Holidays"] = df_final.apply( lambda x: holiday_country(x["CountryCode"], x["Date"]), axis=1)

# convert date to ordinal like
#df_final['Date'] = df_final['Date'].apply(lambda x: x.to_pydatetime().strftime("%Y%m%d"))

# Add a column for new cases only
df_final['NewCases'] = df_final.ConfirmedCases.diff().fillna(0)

# select columns of interest
df_final = df_final[["Date", "ConfirmedCases", "NewCases", "Holidays"]]

In [63]:
print(df_final[1:30])


             Date  ConfirmedCases  NewCases  Holidays
37045  2020-01-02             NaN       0.0         0
37046  2020-01-03             NaN       0.0         0
37047  2020-01-04             NaN       0.0         1
37048  2020-01-05             NaN       0.0         1
37049  2020-01-06             NaN       0.0         0
37050  2020-01-07             NaN       0.0         0
37051  2020-01-08             NaN       0.0         0
37052  2020-01-09             NaN       0.0         0
37053  2020-01-10             NaN       0.0         0
37054  2020-01-11             NaN       0.0         1
37055  2020-01-12             NaN       0.0         1
37056  2020-01-13             NaN       0.0         0
37057  2020-01-14             NaN       0.0         2
37058  2020-01-15             NaN       0.0         0
37059  2020-01-16             NaN       0.0         0
37060  2020-01-17             NaN       0.0         0
37061  2020-01-18             NaN       0.0         1
37062  2020-01-19           

In [11]:
pprint([getattr(holidays, x) for x in holidays.list_supported_countries() if len(x) == 3])

[<class 'holidays.countries.aruba.ABW'>,
 <class 'holidays.countries.angola.AGO'>,
 <class 'holidays.countries.united_arab_emirates.ARE'>,
 <class 'holidays.countries.argentina.ARG'>,
 <class 'holidays.countries.australia.AUS'>,
 <class 'holidays.countries.austria.AUT'>,
 <class 'holidays.countries.burundi.BDI'>,
 <class 'holidays.countries.belgium.BEL'>,
 <class 'holidays.countries.bangladesh.BGD'>,
 <class 'holidays.countries.bulgaria.BLG'>,
 <class 'holidays.countries.belarus.BLR'>,
 <class 'holidays.countries.brazil.BRA'>,
 <class 'holidays.countries.canada.CAN'>,
 <class 'holidays.countries.switzerland.CHE'>,
 <class 'holidays.countries.chile.CHL'>,
 <class 'holidays.countries.colombia.COL'>,
 <class 'holidays.countries.czechia.CZE'>,
 <class 'holidays.countries.germany.DEU'>,
 <class 'holidays.countries.djibouti.DJI'>,
 <class 'holidays.countries.denmark.DNK'>,
 <class 'holidays.countries.dominican_republic.DOM'>,
 <class 'holidays.countries.european_central_bank.ECB'>,
 <class '